# Alice, Bob & Eve: Digital signatures using quantum-safe ML-DSA

The algorithms mentioned in previous parts (RSA, ECDSA) are not quantum-safe. Once Eva will have quantum computer available, she can forge the signatures. Luckily, we already have two post-quantum digital signature algorithms available. Here we'll take a look at the ML-DSA (Module‑Lattice‑based Digital Signature Algorithm), formerly known as CRYSTALS-Dilithium.

This algorithm is available natively in .NET 10. For older versions you can use the `Microsoft.Bcl.Cryptography` package. We will be actually doing this, because current version of .NET Interactive is using .NET 9.

## Hack: Installing the NuGet package 

In regular .NET 10 applications you do not need to install this NuGet package, as everything is part of the runtime itself. However, at time of writing, .NET Interactive (which is used in notebooks) uses version 9.0 and we have to import the package manually:

In [10]:
// Install the Microsoft.Bcl.Cryptography package 
#r "nuget: Microsoft.Bcl.Cryptography"

// Import namespace
using System.Security.Cryptography;

// Show what runtime we're actually on
Console.WriteLine($"Runtime version:  {System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription}");

// Check whether ML-KEM is supported on this platform
Console.WriteLine($"ML-KEM supported: {MLKem.IsSupported}");

Installed Packages Microsoft.Bcl.Cryptography, 10.0.1

Runtime version:  .NET 9.0.11
ML-KEM supported: True


## Alice

As usual, Alice will generate her key pair. Notice that the keys are significantly longer not only than ECDSA, but also than RSA keys. But this is completely different scheme, so it's quite hard to compare them.

The API is kind of similar to the one exposed by `AsymmetricAlgorithm` base class, but the post-quantum algorithm classes are not derived from this class, because the philosophy and usage is different enough.

In [ ]:
using System.Security.Cryptography;

var mldsa = MLDsa.GenerateKey(MLDsaAlgorithm.MLDsa44);
var privateKeyBytes = mldsa.ExportMLDsaPrivateKey();
var publicKeyBytes = mldsa.ExportMLDsaPublicKey();


// Display the keys
Console.WriteLine("Alice's Private Key:");
Console.WriteLine(Convert.ToBase64String(privateKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Private Key (20480 bits):
bXwjHywEG4E6+6At1uTbeKHHuP9/nYGVj8wGi4Gddwnks6amY6RaBDZCoAJrTTuIkLfmuYEri3g3
cdggYY5VVkRS/phAmYz6nlByNRXHfC+DjsDCo4oCbOIoU8p7liXT1fB4+LQLBI/b2VXG8JjOsaDk
2INU/fGxA8/fvwpPyLobsS0SgYETJBEaOY0UgogSsoAYSCLKwmgCwoDUlnECBUAKBQgMpFFDRDAg
NEBKlkzCslGBEk2hhiVYQjEAJWygNI4cIQQMmInjEGkEIQnkwBCkElEZxUDiRJBIQEwjKHGREoWB
CHCUOAICswlDQg1hSFKhEFILxYAhIS3LhEgAEE7DtmRQlgncIAygRAykAgZZMmnAGGKgmIwhGQbM
pE0aCXEjBCkbsAiStGghKCyJyGwZoYSKtISMmI0BpoHZwkkiyUkgIwUCpTHAEnLYMiCZmIQREgbh
JgokpW0kgYBBEBJTGFAAIEHbtABbwJEZslHApFEDAGwJMYBhQCFTFFGCNnEYOIIDlQmkFCJbhGSM
NCpAsA0DAUEbokWDAiAEp4AMJ0IRGFLJIihAhFGRhikREkyDEAyDAonblGELtWQMtjCbyEHgAozR
ApLIokASBXAklZCJkAjalEEBp2AUwSkcI21ANBAhMjBJxlAQEyQcJ0hcNGIgEkFSlCFQFmUUxGxJ
OEIYlwCDMkTQxoVEIE3UwFBhNk4Yw0VZkAURswnaQiBakpHAFA4kkylRBCCbJhBYkHDbsiBACEYc
gA2iklCMtoQkuCQDE1EamS2jNG4TKJEYJyqLSCKAgC0Bp0VjhjCDRGgcySxiyCGhmDADJmRgAGRg
BFIKooHhBojEMIiUBkzTEAIkQHKkBmqRsI2hknBChHERw4VZMkZEFgkjI0njFIpakkkKRCRZJIBK
oGVMRg4jlkDkMiZhBiaZMihDoClbKDIhACAClm3ZJI

In [ ]:
Console.WriteLine("Alice's Public Key ({publicKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(publicKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Public Key (10496 bits):
bXwjHywEG4E6+6At1uTbeKHHuP9/nYGVj8wGi4Gddwl2A5zCweHRuLGPnbAtUYjulA3+1zVthY1L
IgZcUGM+VCGtD9yHfzuK+gaVYxMWuxfm3xCIQPwEHV7pwBb9fBUdJEhNFN0A/htovc7p0LVZkAGD
p54+a/oKJNFVMi+Czh2wnHNuz2CGyo6DonEju7Tdty4nvqfXGEd+6GyCVKU5CJCQ1AHIv60YmfHX
LAg7BfkA+d0lZVLC/CWTncNpxIWH6ato//XFEBC+KCM1W4htH7cL97mP2/MPagzDsT/5/mIfXyCB
whAxDrkQZBAWS6ZyavSjWtztfNI1/Q7faGtCaVf4P/PyT+X8kcMR4TvJScGNxv/isKUlonnbH4Hs
9wmdpae2NTemihjOosmkbKmRxYg2Bh8FA7sBEjOmSOpMDZc4pBPQvKWCgj+ksyRicuG0j9upZKvo
ZAOtkIvFE0/o9dThTijLEGKHovdm20d+oLwmhq1g8IUKShfHOFl3xQlCWSiUfSaH8Mw8xDK3vPhm
XlAZJ/adfLGIdKb0rTLAFLGBYXxP2NFdPHRHHeQABjt8+WYv3pN3QANEeNE4DFE5y5R8mIuVwoxP
uZF6D/K6NYNRRonF1SrLPvaafcVBoOOeMDS/TUONQZPMXw9t/zF/2ny6PljtXhYmCGreBGVSEmiI
deYBoz4R28Yz1wx18751KPhMhLOdila2ffqEL9rJO20f5y6tqsHsbRNco0VCJwqSoa+lu2ssKRSW
CtNQeoHjXR7ZeBOASzSq6EOX21MQ4MJP4djKApSkCN4AkXTvAYNbBZYqfaLsBtzTOjVKxVavm6i9
6SZKoyKPnP+5JI/hBsKrdGZX2xVGpT1UlPg8rY8YVzYJFFJCMj7k2tMPrDS1kjWqUyDf8+jiXB3I
DgPdhmWOicwxf+XLowjRZ6Koe23vyUCpY8KF533Jqqh

Alice will then generate signature of her message. Notice that the signature itself is also smaller, than in case of RSA:

In [13]:
var messageText = "Bob, I love you!";
var messageBytes = System.Text.Encoding.UTF8.GetBytes(messageText);
Console.WriteLine($"Original Message: {messageText}");
Console.WriteLine();

// Sign the message
var signatureBytes = mldsa.SignData(messageBytes);
var signatureText = Convert.ToBase64String(signatureBytes, Base64FormattingOptions.InsertLineBreaks);
Console.WriteLine($"Signature:");
Console.WriteLine(signatureText);

Original Message: Bob, I love you!

Signature:
ipCoVKAj9RwHT8mNG/h9AbOPIJPloqaxpXK56HKPjfhaRAweUmSSNEGCVTlW5FXHsbeu45FQ0qIU
8FkJ/A9rIU+Zj42fe2eET5veTLakmzOW5Yl/pKaRL7iyN+2HWKLBtVLaJpFtDm2KwVpiLT7EX+Aj
Z0YImk7xE+9c+ATqAZor4oPVd8mMSAReS71e+rjWe9yCqTmpUDpDD6cZcm9BoEA5AEBgKQ+HlHt1
Iku6W820iR45AXt90090BBrPVLGYfu8Pz22puK21FhYJcxqLZhr6oQFqp+M6/5ok5bFRVZjl6BMR
cOmG1Lz9dlkvc/V3LCei9Cjd51wYkxh1ywJgnjwDXXVoVlVrN438f7Pp6aXDR742gCjdnu4zqgzr
VHivHdhkGOeSiDxSlzhAKOwq0WULm0nhNbEj4il3G2naTeFxgn1/Ek2GJZ9HfOWjOYdpGrkYlTqE
/PpaWHlveu7akzbZ4kIAux1PwYYtrdgRgOal0vN1dJWaeNr26ykJd3IHOh3QLreJmLc/d2Txdk/b
4tHSFXqCKfJUld1PHWoVr8kssgSnbT3jlqHlnF5lBrvjMPUdt2SjYvBwMt6h7aFWGMA/71MYHaEQ
LFshhf+w8ADxHPOMmWkDE1nbHAyUyHPpA5lUo6WFjTELkPLfPM3tMb2e+Z+ezyuADb403HXHAC54
LWg0MR2KbTf2tMQlRDhbQuO4Ybq9SzAYszFS7SVudmiHotaz2kizV6qNcogkPmjc38+iJ9an5E1T
B0UIgD2I4V8tq/7RvlJQe1eGt837rEVtjSLKJ7T5TeARgMvDRagicKyPkXcxazLLrzJIvpeS5nBx
WS5dqYKXPLAPCOiHbHv3HtyXUbn9fB6rZLj1AnfSeRwMTDB3EbSkLAs2zgUEBcEVBxz8YDRzHFY/
CmDrWFe89Uew22MyJmabxZkSd3JBm

## Bob

In [14]:
// Function to verify signature
bool VerifySignature(byte[] publicKey, string message, byte[] signature) {
    // Create MLDsa instance for verification using the public key
    using var mldsaVerify = MLDsa.ImportMLDsaPublicKey(MLDsaAlgorithm.MLDsa44, publicKey);

    // Convert message to bytes
    var messageBytesVerify = System.Text.Encoding.UTF8.GetBytes(message);

    // Verify the signature
    return mldsaVerify.VerifyData(messageBytesVerify, signature);
}

Console.WriteLine($"Is 'Bob, I love you!' valid? {VerifySignature(publicKeyBytes, "Bob, I love you!", signatureBytes)}");
Console.WriteLine($"Is 'Bob, I hate you!' valid? {VerifySignature(publicKeyBytes, "Bob, I hate you!", signatureBytes)}");

Is 'Bob, I love you!' valid? True
Is 'Bob, I hate you!' valid? False
